In [1]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 1.5 MB/s            


In [2]:
import pandas as pd
import numpy as np
import os
import glob




In [3]:
os.chdir('new-data')


In [4]:
os.listdir()

['.DS_Store',
 '22.8.xlsx',
 'anh-tham',
 'data.zip',
 'fahasa',
 'new-outcome',
 'njv-total.xlsx',
 'sent-phones.xlsx',
 'snappy-total.xlsx']

In [5]:
snp = pd.read_excel('snappy-total.xlsx')

In [7]:
snp.drop(['Unnamed: 0'], inplace=True, axis=1)

In [11]:
snp.drop_duplicates(subset=['To Contact'], inplace=True)

In [12]:
snp['To Contact'].is_unique

True

In [16]:
snp.head()

,To Name,To Contact,To Address,To Province,To District,sent
0,Long,0798310266,khách sạn hải yến đi lên ngã 3 tân cánh rẽ...,Quảng Ninh,Thành phố Cẩm Phả,False
1,Có Phạm Văn,0378136244,"Khánh Bình 05 - Phường Khánh Bình, Thị xã Tân ...",Bình Dương,Thị xã Tân Uyên,False
2,Phiowng Thuý Hà,0379195023,"Cây xăng Cẩm Đông - Phường Cẩm Đông, Thành phố...",Quảng Ninh,Thành phố Cẩm Phả,False
3,Thanh Vân Nguyễn,0936930336,"Sau shop Linh Nguyên - Phường Cẩm Tây, Thành p...",Quảng Ninh,Thành phố Cẩm Phả,True
4,Face Tran Sau,0398498811,"Cẩm phả - Phường Cẩm Đông, Thành phố Cẩm Phả, ...",Quảng Ninh,Thành phố Cẩm Phả,False


In [20]:
isp_maps = {
    70: 'Mobifone',
    79: 'Mobifone',
    77: 'Mobifone',
    76: 'Mobifone',
    78: 'Mobifone',
    32: 'Viettel',
    33: 'Viettel',
    34: 'Viettel',
    35: 'Viettel',
    36: 'Viettel',
    37: 'Viettel',
    38: 'Viettel',
    39: 'Viettel',
    83: 'Vinaphone',
    84: 'Vinaphone',
    85: 'Vinaphone',
    81: 'Vinaphone',
    82: 'Vinaphone',
    97: 'Viettel',
    90: 'Mobifone',
    93: 'Mobifone',
    96: 'Viettel',
    98: 'Viettel',
    91: 'Vinaphone',
    92: 'Vietnammobile',
    94: 'Vinaphone',
    89: 'Mobifone',
    88: 'Vinaphone',
    86: 'Viettel',
    55: 'Vietnammobile',
    56: 'Vietnammobile',
    58: 'Vietnammobile',
    87: 'I-telecom',
    52: 'Vietnammobile'
}


def get_isp_name(maps, string):
    # get isp name from mapping dict
    try:
        return maps.get(int(string[1:3]), np.nan)
    except ValueError:
        return np.nan

In [21]:
snp['ISP'] = snp['To Contact'].astype('str').apply(lambda x: get_isp_name(isp_maps, x))


In [22]:
snp.head()

,To Name,To Contact,To Address,To Province,To District,sent,ISP
0,Long,0798310266,khách sạn hải yến đi lên ngã 3 tân cánh rẽ...,Quảng Ninh,Thành phố Cẩm Phả,False,Mobifone
1,Có Phạm Văn,0378136244,"Khánh Bình 05 - Phường Khánh Bình, Thị xã Tân ...",Bình Dương,Thị xã Tân Uyên,False,Viettel
2,Phiowng Thuý Hà,0379195023,"Cây xăng Cẩm Đông - Phường Cẩm Đông, Thành phố...",Quảng Ninh,Thành phố Cẩm Phả,False,Viettel
3,Thanh Vân Nguyễn,0936930336,"Sau shop Linh Nguyên - Phường Cẩm Tây, Thành p...",Quảng Ninh,Thành phố Cẩm Phả,True,Mobifone
4,Face Tran Sau,0398498811,"Cẩm phả - Phường Cẩm Đông, Thành phố Cẩm Phả, ...",Quảng Ninh,Thành phố Cẩm Phả,False,Viettel


In [23]:
snp.drop(['sent'], inplace=True, axis=1)

In [25]:
snp['To Contact'].is_unique

True

In [26]:
hcm = snp[snp['To Province'] == 'Hồ Chí Minh']

In [27]:
hcm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161809 entries, 9 to 889914
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   To Name      161809 non-null  object
 1   To Contact   161809 non-null  object
 2   To Address   161809 non-null  object
 3   To Province  161809 non-null  object
 4   To District  161809 non-null  object
 5   ISP          161582 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [29]:
hcm['To Contact'].is_unique

True

In [30]:
def split_df(df, chunksize=10000):
    chunks = []
    num_chunks = len(df) // chunksize + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunksize:(i+1)*chunksize])
    return chunks

# chunksize = 10000
# chunks = len(data) // chunksize
chunks = split_df(hcm)



In [32]:
len(chunks)

17

In [ ]:
i = 0
for chunk in chunks:
    chunk.to_excel('hcm-{:02d}.xlsx'.format(i), index=False)
    i += 1